In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import seaborn as sns
sns.set_context('notebook')
sns.set(style="whitegrid", font_scale=1.5)
sns.despine()
sns.set_color_codes()

import matplotlib.pyplot as plt
plt.xlim(0, 1)
plt.ylim(0, None)
from matplotlib.ticker import MaxNLocator
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [20, 5]

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

import vtk
import numpy as np
import nibabel as nib
from six import iteritems
from scipy.stats import norm
from nibabel.affines import apply_affine
from vtk.util import numpy_support as ns
from dipy.tracking.streamline import values_from_volume
from dipy.tracking.metrics import winding

In [2]:
main_folder = '/Users/imag2/Dropbox/IMAG2/MAR/'
dirs = os.listdir(main_folder)

post_op = ["1-4-02", "1-4-10", "1-4-13", "1-4-18", "1-4-20_190715", "1-4-21_190506", "1-4-22_2019", "1-4-25", 
           "1-4-27_190603", "1-4-29_190708", "1-4-32", "1-4-39", "1-4-40", "1-4-53", "1-4-60"]
pre_op = ["1-4-03", "1-4-04_160425", "1-4-05", "1-4-06", "1-4-07", "1-4-11", "1-4-16", "1-4-19", 
          "1-4-20_161017", "1-4-21_161107", "1-4-22_2016", "1-4-26", "1-4-27_170213", "1-4-29_170227", 
          "1-4-33", "1-4-37", "1-4-38", "1-4-41", "1-4-44", "1-4-45_180312", "1-4-45_190506", "1-4-48", 
          "1-4-49", "1-4-51", "1-4-52", "1-4-55", "1-4-56", "1-4-57", "1-4-64"]

filles = ["1-4-03", "1-4-05", " 1-4-11", "1-4-16", "1-4-21_161107", "11-4-27_170213", "1-4-37", "1-4-38" ,
          "1-4-44", "1-4-48", "1-4-52"]
gars = [ "1-4-04", "1-4-06", "1-4-07", "1-4-19", "1-4-20_161017" , "1-4-22_2016", "1-4-26", "1-4-29_170227",
        "1-4-33", "1-4-41", "1-4-45", "1-4-49", "1-4-51", "1-4-55", "1-4-56", "1-4-57", "1-4-64"]

haute = ["1-4-04", "1-4-06", "1-4-16", "1-4-20_161017", "1-4-27_170213", "1-4-33", "1-4-41", "1-4-44",
         "1-4-45", "1-4-49", "1-4-51", "1-4-52" "1-4-55", "1-4-64"]
basse = ["1-4-03", "1-4-05", "1-4-07", "1-4-11", "1-4-19", "1-4-21_161107", "1-4-22_2016", "1-4-26",
         "1-4-29_170227", "1-4-37", "1-4-38", "1-4-48", "1-4-56", "1-4-57"]

sans_an_medu = ["1-4-03", "1-4-04", "1-4-05", "1-4-06", "1-4-11", "1-4-16", "1-4-19", "1-4-20_161017",
                "1-4-21_161107", "1-4-22_2016", "1-4-27_170213", "1-4-29_170227", "1-4-33", "1-4-38",
                "1-4-41", "1-4-44", "1-4-48", "1-4-51", "1-4-52", "1-4-55", "1-4-56"]
avec_an_medu = ["1-4-07", "1-4-26", "1-4-45", "1-4-49", "1-4-57", "1-4-64"]

sans_an_sacr = ["1-4-03", "1-4-04", "1-4-05", "1-4-06", "1-4-11", "1-4-19", "1-4-21_161107",
                "1-4-22_201", "1-4-26", "1-4-27", "1-4-29", "1-4-48", "1-4-51", "1-4-52", "1-4-56", "1-4-57"]
avec_an_sacr = ["1-4-07", "1-4-16", "1-4-20_161017", "1-4-33", "1-4-38", "1-4-41", "1-4-44", "1-4-45",
                "1-4-49", "1-4-55", "1-4-64"]

sans_an_tot = ["1-4-03", "1-4-04", "1-4-05", "1-4-06", "1-4-11", "1-4-19", "1-4-21_161107",
               "1-4-22_2016", "1-4-27", "1-4-29_170227", "1-4-48", "1-4-51", "1-4-52", "1-4-56"]
avec_an_tot = ["1-4-07", "1-4-16", "1-4-20_161017", "1-4-26", "1-4-33", "1-4-38", "1-4-41", "1-4-44",
               "1-4-45", "1-4-49", "1-4-55", "1-4-57", "1-4-64"]

temoins = ["1-2-04", "1-2-05", "1-2-07", "1-2-12", "1-2-14", "1-2-19", "1-2-23", "1-2-28_2016", "1-2-33_2017",
          "1-2-37", "1-2-46", "1-2-51", "1-2-52", "1-2-58", "1-5-02", "1-5-03","1-5-04","1-5-05","1-5-06",
          "1-5-07"]

In [3]:
def load_nii(fname):
    img = nib.load(fname)
    return img.get_fdata(), img.affine

def read_vtk(filename):
    if filename.endswith('xml') or filename.endswith('vtp'):
        polydata_reader = vtk.vtkXMLPolyDataReader()
    else:
        polydata_reader = vtk.vtkPolyDataReader()

    polydata_reader.SetFileName(filename)
    polydata_reader.Update()

    polydata = polydata_reader.GetOutput()

    return vtkpolydata_to_tracts(polydata)

def vtkpolydata_to_tracts(polydata):
    result = {'lines': ns.vtk_to_numpy(polydata.GetLines().GetData()),
              'points': ns.vtk_to_numpy(polydata.GetPoints().GetData()),
              'numberOfLines': polydata.GetNumberOfLines()
             }

    data = {}
    if polydata.GetPointData().GetScalars():
        data['ActiveScalars'] = polydata.GetPointData().GetScalars().GetName()
    if polydata.GetPointData().GetVectors():
        data['ActiveVectors'] = polydata.GetPointData().GetVectors().GetName()
    if polydata.GetPointData().GetTensors():
        data['ActiveTensors'] = polydata.GetPointData().GetTensors().GetName()

    for i in range(polydata.GetPointData().GetNumberOfArrays()):
        array = polydata.GetPointData().GetArray(i)
        np_array = ns.vtk_to_numpy(array)
        if np_array.ndim == 1:
            np_array = np_array.reshape(len(np_array), 1)
        data[polydata.GetPointData().GetArrayName(i)] = np_array

    result['pointData'] = data

    tracts, data = vtkpolydata_dictionary_to_tracts_and_data(result)
    return tracts, data

def vtkpolydata_dictionary_to_tracts_and_data(dictionary):
    dictionary_keys = {'lines', 'points', 'numberOfLines'}
    if not dictionary_keys.issubset(dictionary):
        raise ValueError("Dictionary must have the keys lines and points" + repr(dictionary))

    tract_data = {}
    tracts = []

    lines = np.asarray(dictionary['lines']).squeeze()
    points = dictionary['points']

    actual_line_index = 0
    number_of_tracts = dictionary['numberOfLines']
    original_lines = []
    for _ in range(number_of_tracts):
        tracts.append(points[lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1]])
        original_lines.append(
            np.array(lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1], copy=True))
        actual_line_index += lines[actual_line_index] + 1

    if 'pointData' in dictionary:
        point_data_keys = [it[0] for it in iteritems(dictionary['pointData']) if isinstance(it[1], np.ndarray)]

        for k in point_data_keys:
            array_data = dictionary['pointData'][k]
            if k not in tract_data:
                tract_data[k] = [array_data[f] for f in original_lines]
            else:
                np.vstack(tract_data[k])
                tract_data[k].extend([array_data[f] for f in original_lines[-number_of_tracts:]])

    return tracts, tract_data

In [4]:
def get_hist(bundle, metric):
    def streamlines_mapvolume(streamlines, volume, affine):
        mapping = values_from_volume(volume, streamlines, affine)
        return mapping

    scalar_measurement = streamlines_mapvolume(bundle, metric, affine)
    scalar_measurement = [item for sublist in scalar_measurement for item in sublist]
    return scalar_measurement

def get_length(bundle):
    def _get_length(tract):
        return ((((tract[1:] - tract[:-1]) ** 2).sum(1)) ** .5).sum()
    tracts_lengths = [_get_length(s) for s in bundle]
    return tracts_lengths

def get_shortest(bundle):
    def _get_shortest(tract):
        return ((((tract[0] - tract[-1]) ** 2).sum()) ** .5).sum()
    tracts_shortest = [_get_shortest(s) for s in bundle]
    return tracts_shortest

def get_winding(bundle):
    windings = [winding(s) for s in bundle]
    return windings

In [5]:
def plot_series(serie1, serie2, axes, i):
    try:
        sns.distplot(serie1, rug=True, fit=norm, kde=False, ax = axes[i][0])
        axes[i][0].text(x=0.97, y=0.97, transform=axes[i][0].transAxes, s="Skewness: {:.4f}".format(serie1.skew()),\
                fontweight='demibold', fontsize=20, verticalalignment='top', horizontalalignment='right',\
                backgroundcolor='white', color='xkcd:poo brown')
        axes[i][0].text(x=0.97, y=0.85, transform=axes[i][0].transAxes, s="Kurtosis: {:.4f}".format(serie1.kurt()),\
                fontweight='demibold', fontsize=20, verticalalignment='top', horizontalalignment='right',\
                backgroundcolor='white', color='xkcd:dried blood')
        sns.distplot(serie2, rug=True, fit=norm, kde=False, ax = axes[i][0])
        axes[i][0].text(x=0.97, y=0.77, transform=axes[i][0].transAxes, s="Skewness: {:.4f}".format(serie2.skew()),\
                fontweight='demibold', fontsize=20, verticalalignment='top', horizontalalignment='right',\
                backgroundcolor='white', color='xkcd:poo brown')
        axes[i][0].text(x=0.97, y=0.69, transform=axes[i][0].transAxes, s="Kurtosis: {:.4f}".format(serie2.kurt()),\
                fontweight='demibold', fontsize=20, verticalalignment='top', horizontalalignment='right',\
                backgroundcolor='white', color='xkcd:dried blood')
        sns.boxplot(serie1, ax = axes[i][1],  orient='v')
        sns.boxplot(serie2, ax = axes[i][1],  orient='v')
        sns.violinplot(serie1, ax = axes[i][2], inner="quartiles",  orient='v')
        sns.violinplot(serie2, ax = axes[i][2], inner="quartiles",  orient='v')
        sns.boxenplot(serie1, ax = axes[i][3],  orient='v')
        sns.boxenplot(serie2, ax = axes[i][3],  orient='v')
    except:
        pass

    plt.show;
    
def plot_frame(frame, serie1, serie2, axes, i):
    try:
        sns.distplot(frame[serie1], rug=True, fit=norm, kde=False, ax = axes[i][0])
        sns.distplot(frame[serie2], rug=True, fit=norm, kde=False, ax = axes[i][0])
        sns.boxplot(ax = axes[i][1], data = frame,  orient='v')
        sns.violinplot(data = frame, ax = axes[i][2], inner="quartiles",  orient='v')
        sns.boxenplot(data = frame, ax = axes[i][3],  orient='v')
    except:
        pass

In [ ]:
l5_1 = []
l5_2 = []
s1_1 = []
s1_2 = []
s2_1 = []
s2_2 = []
s3_1 = []
s3_2 = []
s4_1 = []
s4_2 = []

for current_dir in dirs:
    if os.path.isdir(os.path.join(main_folder,current_dir)):
        l5 = []
        s1 = []
        s2 = []
        s3 = []
        s4 = []

        for file in os.listdir(os.path.join(main_folder,current_dir)):
            current_path = os.path.abspath(os.path.join(main_folder, current_dir, file))
            if '.vtk' in current_path.lower() and 'l5' in current_path.lower():
                l5 = read_vtk(current_path)[0]
            elif '.vtk' in current_path.lower() and 's1' in current_path.lower():
                s1 = read_vtk(current_path)[0]
            elif '.vtk' in current_path.lower() and 's2' in current_path.lower():
                s2 = read_vtk(current_path)[0]
            elif '.vtk' in current_path.lower() and 's3' in current_path.lower():
                s3 = read_vtk(current_path)[0]
            elif '.vtk' in current_path.lower() and 's4' in current_path.lower():
                s4 = read_vtk(current_path)[0]
            elif '.nii' in current_path.lower() and 'fa' in current_path.lower():
                FA, affine = load_nii(current_path)
                FA = (FA - np.amin(FA)) / (np.amax(FA) - np.amin(FA))
            elif '.nii' in current_path.lower() and 'md' in current_path.lower():
                MD, _ = load_nii(current_path)
                MD = (MD - np.amin(MD)) / (np.amax(MD) - np.amin(MD))
            elif '.nii' in current_path.lower() and 'zero' in current_path.lower():
                bzero, _ = load_nii(current_path)
                bzero = (bzero - np.amin(bzero)) / (np.amax(bzero) - np.amin(bzero))
        if current_dir in haute and current_dir in sans_an_tot:
            l5_1 += get_hist(l5, MD);
            s1_1 += get_hist(s1, MD);
            s2_1 += get_hist(s2, MD);
            s3_1 += get_hist(s3, MD);
            s4_1 += get_hist(s4, MD);
            #l5_1 += get_winding(l5);
            #s1_1 += get_winding(s1);
            #s2_1 += get_winding(s2);
            #s3_1 += get_winding(s3);
            #s4_1 += get_winding(s4);
        elif current_dir in basse and current_dir in sans_an_tot:
            l5_2 += get_hist(l5, MD);
            s1_2 += get_hist(s1, MD);
            s2_2 += get_hist(s2, MD);
            s3_2 += get_hist(s3, MD);
            s4_2 += get_hist(s4, MD);
            #l5_2 += get_winding(l5);
            #s1_2 += get_winding(s1);
            #s2_2 += get_winding(s2);
            #s3_2 += get_winding(s3);
            #s4_2 += get_winding(s4);

serie_1_name = 'High'
serie_2_name = 'Low'
l5_1 = pd.Series(l5_1, name=serie_1_name)
l5_2 = pd.Series(l5_2, name=serie_2_name)
l5_frame = pd.concat([l5_1, l5_2], axis=1)
s1_1 = pd.Series(s1_1, name=serie_1_name)
s1_2 = pd.Series(s1_2, name=serie_2_name)
s1_frame = pd.concat([s1_1, s1_2], axis=1)
s2_1 = pd.Series(s2_1, name=serie_1_name)
s2_2 = pd.Series(s2_2, name=serie_2_name)
s2_frame = pd.concat([s2_1, s2_2], axis=1)
s3_1 = pd.Series(s3_1, name=serie_1_name)
s3_2 = pd.Series(s3_2, name=serie_2_name)
s3_frame = pd.concat([s3_1, s3_2], axis=1)
s4_1 = pd.Series(s4_1, name=serie_1_name)
s4_2 = pd.Series(s4_2, name=serie_2_name)
s4_frame = pd.concat([s4_1, s4_2], axis=1)

f, axes = plt.subplots(ncols=4, nrows=5, figsize=(25,25), gridspec_kw={'width_ratios': [5, 1, 1, 1]})
axes[0][0].set_title('MD - High / Low - No Anomalies', weight='bold').set_fontsize('30')
plot_frame(l5_frame, serie_1_name, serie_2_name, axes, 0);
plot_frame(s1_frame, serie_1_name, serie_2_name, axes, 1);
plot_frame(s2_frame, serie_1_name, serie_2_name, axes, 2);
plot_frame(s3_frame, serie_1_name, serie_2_name, axes, 3);
plot_frame(s4_frame, serie_1_name, serie_2_name, axes, 4);

f.savefig(os.path.abspath(os.path.join(main_folder, 'md_haute_basse_sa.png')));

/Users/imag2/miniconda3/envs/IMAG2_Utilities/lib/python3.7/site-packages/ipykernel_launcher.py:66: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/imag2/miniconda3/envs/IMAG2_Utilities/lib/python3.7/site-packages/ipykernel_launcher.py:78: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
/Users/imag2/miniconda3/envs/IMAG2_Utilities/lib/python3.7/site-packages/seaborn/distributions.py:200: RuntimeWarning: Mean of empty slice.
  line, = ax.plot(a.mean(), 0)
/Users/imag2/miniconda3/envs/IMAG2_Utilities/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/imag2/miniconda3/envs/IMAG2_Utilities/lib/python3.7/site-packages/numpy/lib/histograms.py:908: